## Sequential Orchestration with Semantic Kernel Agent Framework

![SK-Sequential-Orchestration](SKSequentialOrchestration.png)

### Installing the Dependencies and Libraries

In [ ]:
%pip install semantic-kernel==1.30.0, azure-identity, python-dotenv, azure-ai-projects==1.0.0b8

### Importing Utilities

In [ ]:
from azure.identity import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import MessageTextContent
from dotenv import load_dotenv
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails
import asyncio
from typing import Any, Callable, Set, Dict, List, Optional
import jsonref
from azure.ai.projects.models import FunctionTool, ToolSet
import json
from azure.ai.projects.models import CodeInterpreterTool, MessageAttachment

load_dotenv()

model = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL")

project_client = AzureAIAgent.create_client(credential=DefaultAzureCredential(),
                           conn_str=os.getenv("PROJECT_CONNECTION_STRING")
)

### Defining the Weather-Agent with OpenAPI Tool

In [ ]:
with open("./weather_openapi.json", "r") as f:
    openapi_spec = jsonref.loads(f.read())

# Create Auth object for the OpenApiTool 
auth = OpenApiAnonymousAuthDetails()

# Initialize agent OpenApi tool using the read in OpenAPI spec
openapi = OpenApiTool(
    name="get_weather", spec=openapi_spec, description="Retrieve weather information for a location", auth=auth
)

toolset_for_weather_agent = ToolSet()
toolset_for_weather_agent.add(openapi)

weather_agent = await project_client.agents.create_agent(
    model = model,
    name = "Weather-Agent",
    instructions = f"""You are a weather agent and your work is to answer user queries related
                       to weather information using the tools you are equipped with""",
    toolset=toolset_for_weather_agent
)

# [END create_agent_toolset]
print(f"Created agent, ID: {weather_agent.id}")


weather_agent_definition = await project_client.agents.get_agent(agent_id=weather_agent.id)

weather_agent_instance = AzureAIAgent(client = project_client,
                                      definition = weather_agent_definition)



### Defining the Summariser Agent

In [ ]:
summariser_agent = await project_client.agents.create_agent(
    model = model,
    name = "Summariser-Agent",
    instructions = f"""You are a summariser agent meant to summarise the output with clear formatting""",
    
)

# [END create_agent_toolset]
print(f"Created agent, ID: {summariser_agent.id}")


summariser_agent_definition = await project_client.agents.get_agent(agent_id=summariser_agent.id)

summariser__agent_instance = AzureAIAgent(client = project_client,
                                      definition = summariser_agent_definition)

### Sequentially Orchestrating the Agentic Flow

In [ ]:
from semantic_kernel.agents import Agent, ChatCompletionAgent, SequentialOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.contents import ChatMessageContent

agent_messages = []

def agent_response_callback(message: ChatMessageContent) -> None:
    """Observer function to print the messages from the agents and store them."""
    print(f"# {message.name}\n{message.content}")
    agent_messages.append((message.name, message.content))

sequential_orchestration = SequentialOrchestration(
    members=[weather_agent_instance, summariser__agent_instance],
    agent_response_callback=agent_response_callback,
)

# 2. Create a runtime and start it
runtime = InProcessRuntime()
runtime.start()

goal = f"""Help me generate markdowns for better visualization for weather in Mumbai"""

# 3. Invoke the orchestration with a task and the runtime
orchestration_result = await sequential_orchestration.invoke(
    task=goal,
    runtime=runtime,
)

# 4. Wait for the results
value = await orchestration_result.get()
print(f"***** Final Result *****\n{value}")

weather_response = next((content for name, content in agent_messages if name == "Weather-Agent"), None)

# Save the final result and weather agent's response as a markdown file
with open("weather_visualization.md", "w", encoding="utf-8") as md_file:
    md_file.write("# Weather Visualization\n\n")
    if weather_response:
        md_file.write("## Weather-Agent Response\n")
        md_file.write(f"{weather_response}\n\n")
    md_file.write("## Summary\n")
    md_file.write(f"{value}\n")

print("Markdown file 'weather_visualization.md' created.")

# 5. Stop the runtime when idle
await runtime.stop_when_idle()

